# AI Finance Accountant Agent



Build AI Agents to handle various financial tasks

Three agents:
  1. Tool Calling - Websearch
  2. For RAG based query
  3. Deep Research Stock Analysis

##Open-Source Tech Stack:


- Fast OS LLM Inference [Groq]: https://groq.com/
- Agentic Framework [Agno]: https://www.agno.com/
- Vector Database [PgVector]: https://pypi.org/project/pgvector/
- Embeddings [Sentence-transformers]: https://huggingface.co/sentence-transformers
- Containerization [Udocker for Colab]: https://github.com/drengskapur/docker-in-colab
- Websearch [DuckDuckGo]: https://github.com/duckduckgo

## Install Required Packages

In [ ]:
!pip install groq yfinance agno
!pip install groq duckduckgo-search newspaper4k lxml_html_clean agno
!pip install -U sqlalchemy 'psycopg[binary]' pgvector pypdf agno
!pip install udocker
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=9a3af12007f6a13fc18d0e21346f9cf3f8f202c4f331c2ba9f47911d39f7fdc5
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python --version

Python 3.11.11


## Set Environment Variables

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['PHI_API_KEY'] = userdata.get('PHI_API_KEY') #INPUT YOUR AGNO KEY (I had this key before the rebrand)

print("API keys have been set!")


API keys have been set!


In [ ]:
# IF you want to check if the keys are indeed set

# import os
# print("Groq API key set:" if os.getenv("GROQ_API_KEY") else "Groq API key not set")

## Agent 2:
## Knowledge base Query Capability: RAG (Retrieval Augmented Generation)

Since we are using OS Vector Databases, we need to use docker to launch the app and initialize the database

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown  <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


Now we need to initialize the PGVector. It's simpler if you are running from your local terminal but to make it work in the Colab, we need to run the below steps:

In [ ]:
# 1. First install udocker
!udocker --allow-root install

# 2. Kill existing processes and clean up
!pkill -9 -f postgres
!rm -rf /content/pgdata
!udocker --allow-root rm pgvector
!rm -f postgres.log

# 3. Create fresh directory
!mkdir -p /content/pgdata
!chmod -R 777 /content/pgdata

# 4. Pull and create container with correct image path
!udocker --allow-root pull ankane/pgvector
!udocker --allow-root create --name=pgvector ankane/pgvector

# 5. Run the container
!nohup udocker --allow-root run \
    --env="POSTGRES_DB=ai" \
    --env="POSTGRES_USER=ai" \
    --env="POSTGRES_PASSWORD=ai" \
    --env="PGDATA=/var/lib/postgresql/data/pgdata" \
    --volume="/content/pgdata:/var/lib/postgresql/data" \
    --publish="5532:5432" \
    pgvector > postgres.log 2>&1 &

# 6. Connection testing
import time
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def test_db_connection(max_retries=5, wait_time=10):
    db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

    for attempt in range(max_retries):
        try:
            print(f"\nConnection attempt {attempt + 1}/{max_retries}")
            engine = create_engine(db_url)
            with engine.connect() as connection:
                result = connection.execute(text("SELECT version();"))
                version = result.fetchone()[0]
                print("✅ Successfully connected to PostgreSQL!")
                print(f"Server Version: {version}")

                # Test vector extension
                connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
                print("✅ Vector extension ready!")
                return engine
        except OperationalError as e:
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds...")
            print("\nChecking postgres status:")
            !ps aux | grep postgres
            print("\nLatest logs:")
            !tail -n 20 postgres.log
            time.sleep(wait_time)

    return None

# 7. Apply 30 secs sleep time to wait for DB to finish set up before testing for connection
print("Waiting for database to initialize...")
time.sleep(30)

engine = test_db_connection()

if engine:
    print("\n✅ Database is ready for RAG Agent initialization!")
else:
    print("\n❌ Database connection failed. Please check the logs above.")

Error: invalid container id 
Info: downloading layer sha256:f2c967e41f72b294e2b96f25154dda38dbde3603b3be33888fb437147972f24b
Info: downloading layer sha256:c5f09b50002256f9e40253d9f3f34381edbe3ca083eb5ce77ecffc874c087995
Info: downloading layer sha256:2e3723549f1143b2c0381181709301932d6a592d8969d0827c1f0133772dfbe0
Info: downloading layer sha256:7077e54346e0cc4692391042abd0479bb02443892be7c6b1085fe7184caff826
Info: downloading layer sha256:bb153abf380255875eda2f78bb3c853520a77f3175574a91d909b5d6912c75a4
Info: downloading layer sha256:f1a157d7d7b01f004e4e758a97a38a5d10c8ce79348e5b674187a99d4f0cabda
Info: downloading layer sha256:6e662fa63f18991e2026f333e95c9670506a0c891ec82e5593bb613a627c6a96
Info: downloading layer sha256:2c35234636c95a2fed252512bb033c920753cffdd75c796da556a594845c121d
Info: downloading layer sha256:04efcdd3a2a4cbfcbdd1542bb9af0b2ff422f4e7b2cde58bfe8c61521df96056
Info: downloading layer sha256:786562b3be85b223d9577821b409d7147981e9b2c9611e0c5ec8725b1255df43
Info: downl

Define a class **Documemt QA**:
- Initialize a OS sentence-transformer embedding model
- Access the Vector DB path to load the embeddings of the PDF URL passed by user
- Initialize database, ready to run QA

In [ ]:
from typing import Union, List, Tuple
from sentence_transformers import SentenceTransformer
from agno.models.groq import Groq
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase

In [ ]:
class DocumentQA:
    def __init__(self):
        # Initialize embedder
        self.embedder = self._create_embedder()
        # Initialize Groq model
        self.chat_model = Groq(id="llama3-8b-8192")
        # Database URL
        self.db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
        self.current_knowledge_base = None
        self.agent = None

    def _create_embedder(self):
        """Create the embedding model"""
        class EmbeddingModel:
            def __init__(self):
                self.model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
                self.dimensions = 384

            def get_embedding_and_usage(self, text: Union[str, List[str]]) -> Tuple[Union[List[List[float]], List[float]], dict]:
                if isinstance(text, str):
                    embedding = self.model.encode(text)
                    embedding_list = embedding.tolist()
                    usage = {"prompt_tokens": len(text.split()), "total_tokens": len(text.split())}
                    return embedding_list, usage
                else:
                    embeddings = self.model.encode(text)
                    embedding_list = embeddings.tolist()
                    total_tokens = sum(len(t.split()) for t in text)
                    usage = {"prompt_tokens": total_tokens, "total_tokens": total_tokens}
                    return embedding_list, usage

            def get_embedding(self, text: Union[str, List[str]]) -> Union[List[float], List[List[float]]]:
                if isinstance(text, str):
                    return self.model.encode(text).tolist()
                return self.model.encode(text).tolist()

        print("✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!")
        return EmbeddingModel()

    def load_pdf_url(self, url: str, table_name: str = "documents"):
        """Load a PDF from a URL"""
        try:
            # Create PDF URL knowledge base
            self.current_knowledge_base = PDFUrlKnowledgeBase(
                urls=[url],
                vector_db=PgVector(
                    table_name=table_name,
                    db_url=self.db_url,
                    embedder=self.embedder
                ),
            )

            # Initialize the Agent
            self.agent = Agent(
                knowledge=self.current_knowledge_base,
                search_knowledge=True,
                model=self.chat_model
            )

            # Load knowledge base
            print("Loading knowledge base...")
            self.current_knowledge_base.load(recreate=True)
            print("✅ Knowledge base loaded successfully!")

            # Show sample content
            self.show_sample_content()

        except Exception as e:
            print(f"❌ Error loading PDF: {e}")
            import traceback
            print(traceback.format_exc())

    def show_sample_content(self, num_samples: int = 5):
        """Show sample content from the knowledge base"""
        try:
            if not self.current_knowledge_base:
                print("No knowledge base loaded!")
                return

            docs = self.current_knowledge_base.search("")
            print("\nSample documents in knowledge base:")
            print("-" * 50)
            for i, doc in enumerate(docs[:num_samples], 1):
                print(f"\nDocument {i}:")
                if hasattr(doc, 'content'):
                    print(doc.content[:200] + "..." if len(doc.content) > 200 else doc.content)
                elif hasattr(doc, 'text'):
                    print(doc.text[:200] + "..." if len(doc.text) > 200 else doc.text)
        except Exception as e:
            print(f"Error showing samples: {e}")

    def ask(self, question: str):
        """Ask a question about the loaded document"""
        if not self.current_knowledge_base or not self.agent:
            print("Please load a document first!")
            return

        print(f"\nQ: {question}")
        try:
            # Get relevant documents
            relevant_docs = self.current_knowledge_base.search(question)
            print("\nRelevant documents found:", len(relevant_docs) if relevant_docs else 0)

            # Build context from relevant documents
            context = "\n".join([doc.content if hasattr(doc, 'content') else doc.text
                               for doc in relevant_docs])

            # Create a prompt that includes the context
            full_prompt = f"""Based on the following content:{context}
            Question: {question}
            Please provide a detailed answer based ONLY on the information provided above."""

            # Get response with context
            response = self.agent.run(full_prompt)
            print(f"\nA: {response.content}")

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            print(traceback.format_exc())

In [ ]:
RAG_qa = DocumentQA()

✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!


In [ ]:
RAG_qa.load_pdf_url("https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf")

Loading knowledge base...


INFO     Dropping collection

INFO     Table 'ai.documents' does not exist.

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf

INFO     Inserted batch of 100 documents.

INFO     Inserted batch of 31 documents.

INFO     Added 131 documents to knowledge base

✅ Knowledge base loaded successfully!

Sample documents in knowledge base:
--------------------------------------------------

Document 1:
Environmental progress can and should be good for business. We underpin our climate strategy with strong business principles and innovation while harnessing the power of markets to replicate our solut...

Document 2:
Supporting communities worldwide Through our engagement efforts, we work directly with groups and individuals who are addressing environmental injustice in their communities. We evaluate each opportun...

Document 3:
 Environmental Progress Report 104Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 4:
      2024 Environmental Progress Report 102Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 5:
An ambitious goal for 2030: We committed to be carbon neutral for our entire carbon footprint by the end of the decade. Our journey to 2030 centers on

In [ ]:
RAG_qa.ask("Key points in this report? Give in 5 bullets")


Q: Key points in this report? Give in 5 bullets

Relevant documents found: 5

A: Here are 5 key points from the report:

• Apple has established a rigorous chemical safety program, including the Regulated Substances Specification, to ensure the use of chemicals and materials in their products, accessories, and manufacturing processes.

• The company regularly updates and expands its chemical restrictions to surpass current regulatory restrictions, with recent additions including perfluorohexanesulfonic acid (PFHxA), phenol, isopropylated, phosphate (3:1) (PIP 3:1), and skin-sensitizing substances.

• Apple requires suppliers to analyze materials that come into prolonged skin contact with skin according to Apple's requirements and reviews compliance with these requirements, mandating clear restrictions on potentially harmful chemicals in materials.

• The company has conducted toxicological assessments on over 1,600 new materials to proactively evaluate and eliminate potentially harmfu

In [ ]:
RAG_qa.ask("Executive Summary in 100 words")


Q: Executive Summary in 100 words

Relevant documents found: 5

A: Here is a 100-word summary of the 2024 Environmental Progress Report:

Apple's Environmental Progress Report covers their fiscal year 2023. The report highlights Apple's efforts to reduce their carbon footprint and improve sustainability. Apple accounts for their carbon footprint by following international standards, and they have made significant progress towards reducing their direct and indirect greenhouse gas emissions. They have also implemented carbon removals and offsets to maintain carbon neutrality. The report provides detailed breakdowns of Apple's emissions by sector, including energy consumption, manufacturing, transportation, and product use. Apple aims to continue reducing their environmental impact and promote sustainability throughout their supply chain.
